In [132]:
# Dependencies
import tweepy
import json
import numpy as np
from config import consumer_key, consumer_secret, access_token, access_token_secret
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [133]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [134]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [135]:
target_term = ['$NFLX']
\

In [136]:
df_sp500 = pd.read_csv('../Stock_Index_Analysis_Project/SP500_Company_List.csv')
df_sp500.head()

,Ticker,Security,GICS Sector,GICS Sub Industry,Location,Date Added,CIK
0,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094
4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373


In [137]:
Ticker = df_sp500['Ticker']
df = df_sp500.set_index("Ticker")

In [138]:
# Grab 25 tweets
public_tweets = api.search(target_term, count=25, result_type="recent")
print(public_tweets)

{'statuses': [{'created_at': 'Sat Jul 07 13:09:38 +0000 2018', 'id': 1015583622777200641, 'id_str': '1015583622777200641', 'text': 'Netflix INC $NFLX Shareholder Sands Capital Management LLC Has Cut Stake - https://t.co/M3TAsVaH1H', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [{'text': 'NFLX', 'indices': [12, 17]}], 'user_mentions': [], 'urls': [{'url': 'https://t.co/M3TAsVaH1H', 'expanded_url': 'https://www.bibeypost.com/netflix-inc-nflx-shareholder-sands-capital-management-llc-has-cut-stake/', 'display_url': 'bibeypost.com/netflix-inc-nf…', 'indices': [75, 98]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://www.bibeypost.com" rel="nofollow">bibeypost_stok</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 989504590918795265, 'id_str': '989504590918795265', 'name': 'Bibey Post', 'screen_name

In [139]:
public_users = api.search_users(target_term)
public_users

[{'contributors_enabled': False,
  'created_at': 'Fri Mar 29 22:40:45 +0000 2013',
  'default_profile': False,
  'default_profile_image': False,
  'description': 'Welcome to #TheKillingFam, the fan group for current Netflix original program. In memoriam @Resipsa422 by her partner @msmaxey Home Was Us!',
  'entities': {'description': {'urls': []},
   'url': {'urls': [{'display_url': 'fb.me/thekillingfang…',
      'expanded_url': 'http://fb.me/thekillingfangroup',
      'indices': [0, 23],
      'url': 'https://t.co/BkONXBF9Uy'}]}},
  'favourites_count': 22852,
  'follow_request_sent': False,
  'followers_count': 4165,
  'following': False,
  'friends_count': 1870,
  'geo_enabled': False,
  'has_extended_profile': False,
  'id': 1315361642,
  'id_str': '1315361642',
  'is_translation_enabled': False,
  'is_translator': False,
  'lang': 'en',
  'listed_count': 29,
  'location': 'Seattle, Washington',
  'name': 'TheKillingFanGroup',
  'notifications': False,
  'profile_background_color': '

In [140]:
# List Declaration
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
user_account = []
text = []
date = []

In [141]:
# Loop through all tweets
for tweet in public_tweets["statuses"]:

    # Run Vader Analysis on each tweet
    compound = analyzer.polarity_scores(tweet["text"])["compound"]
    pos = analyzer.polarity_scores(tweet["text"])["pos"]
    neu = analyzer.polarity_scores(tweet["text"])["neu"]
    neg = analyzer.polarity_scores(tweet["text"])["neg"]

    # Add each value to the appropriate array
    user_account.append(tweet)
    compound_list.append(compound)
    positive_list.append(pos)
    negative_list.append(neg)
    neutral_list.append(neu)
    text.append(tweet['text'])
    date.append(tweet['created_at'])


In [142]:
# Store the Average Sentiments
sentiment = {"Compound": np.mean(compound_list),
             "Positive": np.mean(positive_list),
             "Neutral": np.mean(negative_list),
             "Negative": np.mean(neutral_list)}

In [143]:
# Print the Sentiments
print(sentiment)
print("")

{'Compound': 0.072972, 'Positive': 0.06432, 'Neutral': 0.03812, 'Negative': 0.89756}



In [146]:
data_pd = pd.DataFrame({"User": user_account, "Compound Point": compound_list, "Positive Point": positive_list, 
                               "Neutral Point": neutral_list, "Negative Point": negative_list, "Text":text, "Date Tweeted":date}, 
                                 columns=("User", "Compound Point",  "Positive Point" ,"Neutral Point", "Negative Point", "Text", "Date Tweeted"))         
data_pd.head()

,User,Compound Point,Positive Point,Neutral Point,Negative Point,Text,Date Tweeted
0,{'created_at': 'Sat Jul 07 13:09:38 +0000 2018...,-0.2732,0.000,0.840,0.160,Netflix INC $NFLX Shareholder Sands Capital Ma...,Sat Jul 07 13:09:38 +0000 2018
1,{'created_at': 'Sat Jul 07 13:09:23 +0000 2018...,0.0000,0.000,1.000,0.000,"As Netflix INC $NFLX Stock Rose, Sands Capital...",Sat Jul 07 13:09:23 +0000 2018
2,{'created_at': 'Sat Jul 07 13:08:26 +0000 2018...,0.0000,0.000,1.000,0.000,Netflix INC $NFLX Stock Rose While Sands Capit...,Sat Jul 07 13:08:26 +0000 2018
3,{'created_at': 'Sat Jul 07 13:02:14 +0000 2018...,-0.2500,0.089,0.776,0.135,"“As you know, they free-ride on connectivity.....",Sat Jul 07 13:02:14 +0000 2018
4,{'created_at': 'Sat Jul 07 13:01:26 +0000 2018...,-0.3871,0.000,0.884,0.116,RT @markbspiegel: @ChangeAlley2018 @beatlesonb...,Sat Jul 07 13:01:26 +0000 2018
5,{'created_at': 'Sat Jul 07 13:01:22 +0000 2018...,-0.2960,0.220,0.550,0.230,@markbspiegel @ChangeAlley2018 @beatlesonbanki...,Sat Jul 07 13:01:22 +0000 2018
6,{'created_at': 'Sat Jul 07 13:00:36 +0000 2018...,0.6892,0.230,0.770,0.000,$SPY calls +120%! Join us in our 💥VIP room 💥an...,Sat Jul 07 13:00:36 +0000 2018
7,{'created_at': 'Sat Jul 07 13:00:08 +0000 2018...,0.0000,0.000,1.000,0.000,New weekly #stock #market review. Don't mis it...,Sat Jul 07 13:00:08 +0000 2018
8,{'created_at': 'Sat Jul 07 12:59:13 +0000 2018...,0.0000,0.000,1.000,0.000,今日の若月は特にかわいかったな〜。\nやっぱり黒髪が似合う。 #musicday,Sat Jul 07 12:59:13 +0000 2018
9,{'created_at': 'Sat Jul 07 12:54:12 +0000 2018...,-0.3871,0.000,0.873,0.127,@ChangeAlley2018 @beatlesonbankin You COMPLETE...,Sat Jul 07 12:54:12 +0000 2018
